### Retraining, using all of the encounters to-date

In [22]:
import datetime
import os
import random
import scipy.stats
from datetime import date
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy.stats as st
from matplotlib.colors import ListedColormap
from scipy.stats import pearsonr, spearmanr
import pickle
from alibi_detect.cd.pytorch import HiddenOutput

from cyclops.utils.file import load_pickle, save_pickle

from cyclops.processors.column_names import (
    ADMIT_TIMESTAMP,
    DISCHARGE_TIMESTAMP,
    ENCOUNTER_ID,
    EVENT_NAME,
    EVENT_TIMESTAMP,
    EVENT_VALUE,
    RESTRICT_TIMESTAMP,
    TIMESTEP,
)
import torch.nn as nn
import torch.optim as optim
from models.temporal.optimizer import Optimizer, EarlyStopper
from drift_detection.drift_detector.clinical_applicator import ClinicalShiftApplicator
from drift_detection.drift_detector.detector import Detector
from drift_detection.drift_detector.experimenter import Experimenter
from drift_detection.drift_detector.plotter import plot_drift_samples_pval
from drift_detection.drift_detector.reductor import Reductor
from drift_detection.drift_detector.tester import DCTester, TSTester
from drift_detection.gemini.constants import DIAGNOSIS_DICT, ACADEMIC, COMMUNITY, HOSPITALS
from drift_detection.gemini.utils import get_use_case_params, impute, prep, import_dataset_hospital
from models.temporal.utils import (
    get_device,
    load_checkpoint,
)
from cyclops.processors.constants import ALL, FEATURES, MEAN, NUMERIC, ORDINAL, STANDARD
from cyclops.processors.feature.vectorize import (
    Vectorized,
    intersect_vectorized,
    split_vectorized,
    vec_index_exp,
)
from drift_detection.drift_detector.utils import get_serving_data, get_temporal_model
from drift_detection.retrainers.cumulative import CumulativeRetrainer

## Get parameters

In [11]:
DATASET = "gemini"
USE_CASE = "mortality"
PATH = "/mnt/nfs/project/delirium/drift_exp/OCT-18-2022/"
TIMESTEPS = 6
AGGREGATION_TYPE = "time"   
ID = SHIFT = input("Select data split: ")
DIAGNOSIS_TRAJECTORY = input("Select diagnosis trajectory to filter on: ") 
HOSPITAL = input("Select hospital to filter on: ") 

splice_map = {
    "hospital_id": HOSPITALS
}

if DIAGNOSIS_TRAJECTORY != "all":
    diagnosis_trajectory = '_'.join(DIAGNOSIS_DICT[DIAGNOSIS_TRAJECTORY])
    ID = ID +"_"+ diagnosis_trajectory
    splice_map["diagnosis_trajectory"] = [diagnosis_trajectory]
    
if HOSPITAL != "all":
    ID = HOSPITAL + "_" + ID 
    splice_map["hospital_id"] = [HOSPITAL]
    
use_case_params = get_use_case_params(DATASET, USE_CASE)

    
MODEL_PATH = os.path.join(
    PATH,
    DATASET,
    USE_CASE,
    "saved_models",
    "simulated_deployment_reweight_positive_lstm_1.pt"
)

Select data split:  simulated_deployment
Select diagnosis trajectory to filter on:  all
Select hospital to filter on:  all


## Get data

In [12]:
random.seed(1)

tab_features = load_pickle(use_case_params.TAB_FEATURES_FILE)

use_case_params = get_use_case_params(DATASET, USE_CASE)

(X_tr_final, y_tr), (X_val_final, y_val), (X_t_final, y_t) = import_dataset_hospital(
    use_case_params.TAB_VEC_COMB, ID, train_frac=0.8, shuffle=True,
)

2023-01-31 15:57:28,918 INFO cyclops.utils.file - Loading pickled data from /mnt/nfs/project/delirium/drift_exp/OCT-18-2022/gemini/mortality/./data/tab_features.pkl
2023-01-31 15:57:28,939 INFO cyclops.utils.file - Loading pickled data from /mnt/nfs/project/delirium/drift_exp/OCT-18-2022/gemini/mortality/./data/4-final/aligned_comb_train_X_simulated_deployment.pkl
2023-01-31 15:57:29,512 INFO cyclops.utils.file - Loading pickled data from /mnt/nfs/project/delirium/drift_exp/OCT-18-2022/gemini/mortality/./data/4-final/aligned_comb_train_y_simulated_deployment.pkl
2023-01-31 15:57:29,557 INFO cyclops.utils.file - Loading pickled data from /mnt/nfs/project/delirium/drift_exp/OCT-18-2022/gemini/mortality/./data/4-final/aligned_comb_val_X_simulated_deployment.pkl
2023-01-31 15:57:29,681 INFO cyclops.utils.file - Loading pickled data from /mnt/nfs/project/delirium/drift_exp/OCT-18-2022/gemini/mortality/./data/4-final/aligned_comb_val_y_simulated_deployment.pkl
2023-01-31 15:57:29,694 INFO cy

## Create data streams

In [13]:
START_DATE = date(2019, 1, 1)
END_DATE = date(2020, 8, 1)

In [14]:
print("Get target data streams...")
data_streams_filepath = os.path.join(
    PATH,
    DATASET,
    USE_CASE,
    "drift",
    "data_streams_"+ID+"_"+str(START_DATE)+"_"+str(END_DATE)+".pkl"
)
data_streams_filepath

if os.path.exists(data_streams_filepath):
    data_streams = load_pickle(data_streams_filepath)
else:
    tab_vectorized = load_pickle(use_case_params.TAB_VECTORIZED_FILE)
    comb_vectorized = load_pickle(use_case_params.COMB_VECTORIZED_FILE)


    ids = tab_features.slice(splice_map)
    tab_vectorized = tab_vectorized.take_with_index(ENCOUNTER_ID, ids)
    # intersect tabular and temporal vectors of source data
    tab_vectorized, comb_vectorized = intersect_vectorized(
        [tab_vectorized, comb_vectorized], axes=ENCOUNTER_ID
    )
    numeric_features = tab_features.features_by_type(NUMERIC)    
    normalizer_map = {feat: STANDARD for feat in numeric_features}
    comb_vectorized.add_normalizer(
        EVENT_NAME,
        normalization_method=STANDARD,
    )
    X, y = comb_vectorized.split_out(EVENT_NAME, use_case_params.TEMP_TARGETS)
    X = impute(X)
    X.fit_normalizer()
    X.normalize()

    X_final = prep(X)
    ind = pd.MultiIndex.from_product(
            [X.indexes[1], range(6)], names=[ENCOUNTER_ID, TIMESTEP]
    )
    X_final = pd.DataFrame(X_final.reshape(X_final.shape[0]*X_final.shape[1], X_final.shape[2]),index=ind, columns=X.indexes[2])
    y_final = y.data.reshape(y.data.shape[1],y.data.shape[3])

    data_streams = get_serving_data(
        X_final,
        y_final,
        tab_features.data,
        START_DATE,
        END_DATE,
        stride=1,
        window=1,
        encounter_id="encounter_id",
        admit_timestamp="admit_timestamp",
    )
    save_pickle(data_streams, data_streams_filepath)

2023-01-31 15:57:31,469 INFO cyclops.utils.file - Loading pickled data from /mnt/nfs/project/delirium/drift_exp/OCT-18-2022/gemini/mortality/drift/data_streams_simulated_deployment_2019-01-01_2020-08-01.pkl


Get target data streams...


## Get shift detector

In [15]:
DR_TECHNIQUE = "BBSDs_trained_LSTM"
TESTER_METHOD = "ks"
THRESHOLD = 0.01
UPDATE_REF=25000

print("Get Shift Reductor...")
reductor = Reductor(
    dr_method=DR_TECHNIQUE,
    model_path=MODEL_PATH,
    n_features=X_tr_final.shape[2],
    var_ret=0.8,
)

tstesters=["lk", "lsdd", "mmd", "tabular", "ctx_mmd","chi2", "fet", "ks" ]
dctesters =["spot_the_diff", "classifier", "classifier_uncertainty"]
CONTEXT_TYPE=None
REPRESENTATION=None

if TESTER_METHOD in tstesters:
    tester = TSTester(
        tester_method=TESTER_METHOD,
    )
    if TESTER_METHOD == "ctx_mmd":
        CONTEXT_TYPE = input("Select context type: ")
        
elif TESTER_METHOD in dctesters:
    MODEL_METHOD = input("Select model method: ")
    tester = DCTester(
        tester_method=TESTER_METHOD,
        model_method=MODEL_METHOD,
    )
        
    if TESTER_METHOD == "lk":
        REPRESENTATION = input("Select learned kernel representation: ")
else:
    print("Tester method invalid or not supported.")
    
print("Get Shift Detector...")
detector = Detector(
    reductor=reductor,
    tester=tester
)

detector.fit(
        X_val_final,
        backend="pytorch",
        device = "cuda",
        model_path = MODEL_PATH,
        batch_size=32,
        verbose=0,
        alternative="greater",
        correction="bonferroni",
        input_dim=X_tr_final.shape[2],
        update_x_ref={'last': UPDATE_REF}
)

Get Shift Reductor...
Loading model...
Get Shift Detector...


## Retrain

In [16]:
retrain = "update"
model_name = "lstm"
output_dim = 1
input_dim = X_tr_final.shape[2]
hidden_dim = 64
layer_dim = 2
dropout = 0.2
last_timestep_only = False
device = get_device()

model_params = {
    "device": device,
    "input_dim": input_dim,
    "hidden_dim": hidden_dim,
    "layer_dim": layer_dim,
    "output_dim": output_dim,
    "dropout_prob": dropout,
    "last_timestep_only": last_timestep_only,
}

if model_name in ["rnn", "gru", "lstm"]:
    model = get_temporal_model(model_name, model_params).to(device)

    if retrain == "update":
        print("Update.")
        model, opt, _ = load_checkpoint(MODEL_PATH, model)
        n_epochs = 1
        opt.clipping_value = 1
    elif retrain == "retrain":
        print("Retrain.")
        n_epochs = 64
        learning_rate = 2e-3
        weight_decay = 1e-6
        clipping_value = 1
        reweight_positive = (y_tr == 0).sum() / (y_tr == 1).sum() * 2
        loss_fn = nn.BCEWithLogitsLoss(reduction="none")
        optimizer = optim.Adagrad(
            model.parameters(), lr=learning_rate, weight_decay=weight_decay
        )
        lr_scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=128, gamma=0.5)
        activation = nn.Sigmoid()
        earlystopper = EarlyStopper(patience=3, min_delta=0)
        opt = Optimizer(
            model=model,
            loss_fn=loss_fn,
            optimizer=optimizer,
            activation=activation,
            lr_scheduler=lr_scheduler,
            reweight_positive=reweight_positive,
            earlystopper = earlystopper,
            clipping_value = clipping_value
        )
    else:
        print("Do nothing.")
# elif model_name == "gbt":
#     with open(model_path, "rb") as f:
#         model = pickle.load(f)
else:
    print("Unsupported model")

Update.


In [17]:
SAMPLE = 1000
STAT_WINDOW = 14
LOOKUP_WINDOW = 0
STRIDE = 1
THRESHOLD = 0.01

In [21]:
all_runs = []
for i in range(0, 5):
    random.seed(i)
    np.random.seed(i)
    
    detector = Detector(
        reductor=reductor,
        tester=tester
    )

    detector.fit(
        X_val_final,
        backend="pytorch",
        device = "cuda",
        model_path = MODEL_PATH,
        batch_size=32,
        verbose=0,
        alternative="greater",
        correction="bonferroni",
        input_dim=X_tr_final.shape[2],
        update_x_ref={'last': UPDATE_REF}
    )
    
    model, opt, _ = load_checkpoint(MODEL_PATH, model)
    
    retrainer = CumulativeRetrainer(
        shift_detector=detector, 
        optimizer=opt, 
        model=model, 
        model_name=model_name
    )
    
    results = retrainer.retrain(
        data_streams=data_streams,
        sample=SAMPLE,
        stat_window=STAT_WINDOW,
        lookup_window=LOOKUP_WINDOW,
        stride=STRIDE,
        model_path=MODEL_PATH,
        threshold=THRESHOLD,
        n_epochs = n_epochs,
        verbose = 1,
        correct_only=0,
        positive_only=1,
    )

    all_runs.append(results)
    pvals_test = results["p_val"]
    mean = np.mean([i for i in pvals_test if i < THRESHOLD] )
    ci = st.t.interval(
        0.95,
        len([i for i in pvals_test if i < THRESHOLD]) - 1,
        loc=np.mean([i for i in pvals_test if i < THRESHOLD]),
        scale=st.sem([i for i in pvals_test if i < THRESHOLD]),
    )
    print(len([i for i in pvals_test if i < THRESHOLD]), " alarms with avg p-value of ", mean, ci)

save_pickle(all_runs, os.path.join(PATH, DATASET,USE_CASE, "_".join(["cumulative",ID, "statwindow"+str(STAT_WINDOW),"update"+str(UPDATE_REF),"epoch"+str(n_epochs),"sample"+str(SAMPLE),"threshold"+str(THRESHOLD),"retraining",retrain,"positiveonly"])+".pkl"))


  0%|          | 0/534 [00:00<?, ?it/s]

Calibrating drift detector...



  9%|▉         | 50/534 [00:15<02:30,  3.21it/s]

P-value below threshold for  2019-02-07 - 2019-02-21 	P-Value:  6.785589e-13
Retrain  lstm  on:  2019-01-02 - 2019-02-07
[1/1] Training loss: 1.5813	                 Validation loss: 1.6623
P-value below threshold for  2019-02-08 - 2019-02-22 	P-Value:  2.0544115e-12
Retrain  lstm  on:  2019-01-02 - 2019-02-08
[1/1] Training loss: 1.6073	                 Validation loss: 1.5812
P-value below threshold for  2019-02-09 - 2019-02-23 	P-Value:  4.538124e-11
Retrain  lstm  on:  2019-01-02 - 2019-02-09
[1/1] Training loss: 1.7297	                 Validation loss: 1.4130
P-value below threshold for  2019-02-10 - 2019-02-24 	P-Value:  5.7628995e-11
Retrain  lstm  on:  2019-01-02 - 2019-02-10
[1/1] Training loss: 1.4691	                 Validation loss: 1.9585
P-value below threshold for  2019-02-11 - 2019-02-25 	P-Value:  5.948557e-10
Retrain  lstm  on:  2019-01-02 - 2019-02-11
[1/1] Training loss: 1.9028	                 Validation loss: 1.4171
P-value below threshold for  2019-02-12 - 2019-0


 28%|██▊       | 150/534 [02:17<06:00,  1.07it/s]

...


 84%|████████▍ | 450/534 [10:12<02:32,  1.82s/it]

P-value below threshold for  2020-05-30 - 2020-06-13 	P-Value:  0.0056656133
Retrain  lstm  on:  2019-01-02 - 2020-05-30
[1/1] Training loss: 1.7663	                 Validation loss: 1.7287


 84%|████████▍ | 450/534 [10:54<02:02,  1.45s/it]
2023-01-31 16:55:39,872 INFO cyclops.utils.file - Pickling data to /mnt/nfs/project/delirium/drift_exp/OCT-18-2022/gemini/mortality/cumulative_simulated_deployment_statwindow14_update25000_epoch1_sample1000_threshold0.01_retraining_update_positiveonly.pkl


98  alarms with avg p-value of  0.0024483092 (0.0018642539342703737, 0.0030323644915757264)


'/mnt/nfs/project/delirium/drift_exp/OCT-18-2022/gemini/mortality/cumulative_simulated_deployment_statwindow14_update25000_epoch1_sample1000_threshold0.01_retraining_update_positiveonly.pkl'